# Model Intercomparison

In [ ]:
import sys
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import geopandas
import matplotlib
import matplotlib.pyplot as plt

sys.path.append('..')

from forcing import Forcing
from real_geometry import RealGeometry, glaciers
from ideal_geometry import IdealGeometry
from Plume import PlumeModel
from PICO import PicoModel
from PICOP import PicoPlumeModel
from Simple import SimpleModels

%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

## Experiment 1: melt rated for reference setups `test1` and `test3`

In [ ]:
# 1D plume model
N = 101
pdict = {'x':np.linspace(0,1e5,N), 'draft':np.linspace(-1000,-500,N), 'Ta':0, 'Sa':34}  # [m]
dp = IdealGeometry('plume1D', pdict).create()
ds_ = PlumeModel(dp).compute_plume(full_nondim=True)
plt.plot(ds_.x/1e3, ds_.m, label='reference 1D')
for i, testcase in enumerate(['test1', 'test3']):
    dp = IdealGeometry(testcase).create()
    ds = PlumeModel(dp).compute_plume()
    plt.plot(ds.x/1e3,ds.m.mean('y')+i, label=testcase)
plt.legend()

In [ ]:
models = ['Plume', 'PICO', 'PICOP']
kw = dict(vmin=0, vmax=50, shading='auto', cmap='plasma')

f, ax = plt.subplots(3,len(models)+1, figsize=(12,9), sharey=True, sharex=True)
ax[0,0].set_title('draft')
for i, testcase in enumerate(['test1', 'test2', 'test3']):
#     print('\n', testcase, '\n')
    ds = IdealGeometry(testcase).create()
    ax[i,0].pcolormesh(ds.x/1e3, ds.y/1e3, ds.draft, shading='auto', vmin=-1000, vmax=-500)
    ax[i,0].set_ylabel('y  [km]')
    for j, model in enumerate(models):
        if i==0:  ax[0,j+1].set_title(model)
        if model=='Plume':
            results = PlumeModel(ds).compute_plume()
            ax[i,j+1].pcolormesh(results.x/1e3, results.y/1e3, results.m, **kw)
            melt = results.m.mean().values
        elif model=='PICO':
            _, results = PicoModel(name=testcase).compute_pico()
            ax[i,j+1].pcolormesh(results.x/1e3, results.y/1e3, results.melt, **kw)
            melt = results.melt.mean().values
#             print(f'({results.mk[0].values:.1f} from within PICO)')            
        elif model=='PICOP':
            _, _, results = PicoPlumeModel(name=testcase, n=3).compute_picop()
            ax[i,j+1].pcolormesh(results.x/1e3, results.y/1e3, results.m, **kw)
            melt = results.m.mean().values
        ax[i,j+1].text(.95,.9, f'{melt:.2f} m/yr', transform=ax[i,j+1].transAxes, ha='right', color='w', fontsize=12)
            
for j in range(4):
    ax[-1,j].set_xlabel('x  [km]')


In [ ]:
models = ['Simple', 'Plume', 'PICO', 'PICOP','Sheet']
kw = dict(vmin=0, vmax=50, shading='auto', cmap='plasma')

f, ax = plt.subplots(2,len(models)+3, figsize=(15,5), sharey=True, sharex=True, constrained_layout=True)
ax[0,0].set_title('draft')
for i, testcase in enumerate(['test1', 'test3']):  # can test with 'test2' to check equivalence 
#     print('\n', testcase, '\n')
    ds = IdealGeometry(testcase).create()
    ds = Forcing(ds).constant()  # default Ta=0, Sa=34
    ax[i,0].pcolormesh(ds.x/1e3, ds.y/1e3, ds.draft, shading='auto', vmin=-1000, vmax=-500)
    ax[i,0].set_ylabel('y  [km]')
    for j, model in enumerate(models):
        if i==0:
            if j==0:
                ax[0,1].set_title(r'M$_{lin}$')
                ax[0,2].set_title(r'M$_{quad}$')
                ax[0,3].set_title(r'M$_{+}$')
            else:
                ax[0,j+3].set_title(model)
                
        if model=='Simple':
            results = SimpleModels(ds).compute()
            ax[i,j+1].pcolormesh(results.x/1e3, results.y/1e3, results.Ml, **kw)
            ax[i,j+2].pcolormesh(results.x/1e3, results.y/1e3, results.Mq, **kw)
            ax[i,j+3].pcolormesh(results.x/1e3, results.y/1e3, results.Mp, **kw)
            ax[i,j+1].text(.95,.9, f'{results.Ml.mean().values:.2f} m/yr', transform=ax[i,j+1].transAxes, ha='right', color='w', fontsize=12)
            ax[i,j+2].text(.95,.9, f'{results.Mq.mean().values:.2f} m/yr', transform=ax[i,j+2].transAxes, ha='right', color='w', fontsize=12)
            ax[i,j+3].text(.95,.9, f'{results.Mp.mean().values:.2f} m/yr', transform=ax[i,j+3].transAxes, ha='right', color='w', fontsize=12)
        elif model=='Plume':
            results = PlumeModel(ds).compute_plume()
            ax[i,j+3].pcolormesh(results.x/1e3, results.y/1e3, results.m, **kw)
            melt = results.m.mean().values
        elif model=='PICO':
            _, results = PicoModel(ds).compute_pico()
            ax[i,j+3].pcolormesh(results.x/1e3, results.y/1e3, results.melt, **kw)
            melt = results.melt.mean().values
#             print(f'({results.mk[0].values:.1f} from within PICO)')            
        elif model=='PICOP':
            _, _, results = PicoPlumeModel(ds).compute_picop()
            ax[i,j+3].pcolormesh(results.x/1e3, results.y/1e3, results.m, **kw)
            melt = results.m.mean().values
        elif model=='Sheet':
            if testcase=='test2': continue
            results = xr.open_dataset(f'../../results/Sheet/Sheet_T0_S34_{testcase}.nc')
            ax[i,j+3].pcolormesh(results.x/1e3, results.y/1e3, results.melt*3600*24*365, **kw)
            melt = results.melt.mean().values*3600*24*365
        if j>0:
            ax[i,j+3].text(.95,.9, f'{melt:.2f} m/yr', transform=ax[i,j+3].transAxes, ha='right', color='w', fontsize=12)
            
for j in range(8):
    ax[-1,j].set_xlabel('x  [km]')


### Amundsen Sea

In [ ]:
proj = ccrs.SouthPolarStereo(true_scale_latitude=-71)
def fn_poly(glacier):  return f'../../data/mask_polygons/{glacier}_polygon.geojson'
x3, _, _, y4 = geopandas.read_file(fn_poly('PineIsland'), crs='espg:3031').total_bounds
_, y3, x4, _ = geopandas.read_file(fn_poly('Dotson')    , crs='espg:3031').total_bounds

import matplotlib.ticker as mticker

models = ['Simple','Plume','PICO']
n = len(models)+1

kw = norm=dict(vmin=0.1, vmax=10**2.5, norm=matplotlib.colors.LogNorm(vmin=.1, vmax=10**2.5), cmap='inferno', transform=ccrs.PlateCarree())
f = plt.figure(figsize=(15,9))
(x1,x2,y1,y2) = (x3,x4,y3-1e4,y4+2e4)
shelves = ['PineIsland','Thwaites','Dotson']
for s, shelf in enumerate(shelves):
    (x,y) = [(.65,.88),(.05,.55),(.05,.2)][s]
    name = ['Pine\nIsland','Thwaites','Dotson/\nCrosson'][s]
    
    ds = RealGeometry(shelf).create()
    ds['dgrl'] = ds.dglm.copy()
    ds = Forcing(ds).tanh(Tdeep=1,ztcl=-600)
    lon, lat = ds.lon, ds.lat
    for j in range(n):
        title = ['draft',r'Simple M$_{+}$','Plume','PICO'][j]
        ax = f.add_axes([j/n,.08,.99/n,.89], projection=proj)
        ax.set_title(title, fontsize=16)
        ax.set_extent([x1,x2,y1,y2], crs=proj)
        ax.coastlines(linewidth=.5)
        
        if j in [0,1]:   # draft + Simple
            if j==0:
                draft = ax.pcolormesh(lon, lat, ds.draft, 
                                      vmin=-1600, vmax=-200,
                                      transform=ccrs.PlateCarree(), cmap='viridis')
            elif j==1:
                results = SimpleModels(ds).compute()
                im = ax.pcolormesh(lon, lat, results.Mp, **kw)
                melt = results.Mp.mean().values
        elif j==2: # Plume
            results = PlumeModel(ds).compute_plume()
            im = ax.pcolormesh(lon, lat, results.m, **kw)
            melt = results.m.mean().values
            
        elif j==3: # PICO
            _, results = PicoModel(ds).compute_pico()
            im = ax.pcolormesh(lon, lat, results.melt, **kw)
            melt = results.melt.mean().values
            
        gl = ax.gridlines()
        gl.xlocator = mticker.FixedLocator(np.arange(-180,179,5))
        gl.ylocator = mticker.FixedLocator(np.arange(-89,89))
        if j==0:  ax.text(x, y, name, weight='bold', transform=ax.transAxes, fontsize=14)
        else:     ax.text(x, y, f'{melt:.1f} m/yr', transform=ax.transAxes, fontsize=14)
cax1 = f.add_axes([0.01,.05,.23,.02])
plt.colorbar(draft, cax=cax1, orientation='horizontal', label='draft [m]')
cax2 = f.add_axes([0.27,.05,.7,.02])
plt.colorbar(im, cax=cax2, orientation='horizontal', label='melt rate [m/yr]')
#         if j==2:  ax.text(x, y, f'{dsP.mk[0].values:.2f} m/yr', transform=ax.transAxes)

In [ ]:
ds.draft.max()

In [ ]:
f = plt.figure()
ax = f.add_axes([j/n,.01,.99/n,.98], projection=proj)
# ax.pcolormesh(lon, lat, results.melt, **kw)
ax.pcolormesh(lon, lat, results.melt,
#               vmin=0.1, vmax=10**2.5,
#               norm=matplotlib.colors.LogNorm(vmin=.1, vmax=10**2.5),
              cmap='inferno', transform=ccrs.PlateCarree())

In [ ]:
N = 100
X, Y = np.mgrid[-3:3:complex(0, N), -2:2:complex(0, N)]

# A low hump with a spike coming out of the top right.  Needs to have
# z/colour axis on a log scale so we see both hump and spike.  linear
# scale only shows the spike.
Z1 = np.exp(-(X)**2 - (Y)**2)
Z2 = np.exp(-(X * 10)**2 - (Y * 10)**2)
Z = Z1 + 50 * Z2

fig, ax = plt.subplots(2, 1)

pcm = ax[0].pcolor(X, Y, Z,
                   norm=matplotlib.colors.LogNorm(vmin=.1, vmax=10**2.5), cmap='inferno')
fig.colorbar(pcm, ax=ax[0], extend='max')

## temperature dependence

In [ ]:
# # from Erwin's runs 2.11.20
# sheet = [[ 2.5686432,   6.18703265, 10.25362111, 14.77641703, 19.75861614, 25.19942571, 31.0948176,  37.43819347, 44.22095555],
#          [ 2.94456839,  7.12903415, 11.71694807, 16.71844499, 22.14056505, 27.98734453, 34.25992276, 40.95688662, 48.0747288 ]]

In [ ]:
# from Erwin's runs 3.12.20
sheet = [[ 0.27507259, 1.37829108, 3.50884363, 6.6422587 , 10.694901  , 15.58477964, 21.24315468, 27.61924939, 34.6677999 ],
         [ 0.26924769, 1.3815549 , 3.52330232, 6.68378076, 10.78488732, 15.73930742, 21.46880014, 27.90813137, 34.99945119]]

In [ ]:
f, ax = plt.subplots(1,2,figsize=(10,4), sharey=True, constrained_layout=True)
ax[0].set_ylabel('mean melt rate [m/yr]')
Tas = np.arange(-2,2.5,.5)
for i, testcase in enumerate(['test1', 'test3']):
    ref = ax[i].axvspan(-.15,.15, alpha=.1, color='purple', label='reference')
    sm = ax[i].scatter(np.arange(-2,2.1,.5), sheet[i], c='k', label='Sheet')
    lb, = ax[i].plot(Tas,  7*(Tas+2), c='lightgrey', label='7 m/yr/K')
    ub, = ax[i].plot(Tas, 16*(Tas+2), c='darkgrey', label='16 m/yr/K')
    ax[i].set_title(testcase)
    ax[i].set_xlabel(r'ambient temperature [$^\circ$C]')
    for T in Tas:
        handles = []
        for j, model in enumerate(models):
            if model=='Plume':
                c = 'C0'
                pdict={'Ta':T}
                ds = IdealGeometry(name=testcase, pdict=pdict).create()
                results = PlumeModel(ds).compute_plume()
                melt = results.m.mean().values
            elif model=='PICO':
                c = 'C1'
                _, results = PicoModel(name=testcase, Ta=T).compute_pico()
                melt = results.melt.mean().values
            elif model=='PICOP':
                c = 'C2'
                _, _, results = PicoPlumeModel(name=testcase, Ta=T, n=3).compute_picop()
                melt = results.m.mean().values
            s = ax[i].scatter(T, melt, c=c, label=model)
            handles.append(s)
    ax[i].legend(handles=[ref]+handles[:3]+[sm,lb,ub])

In [ ]:
models

In [ ]:
f, ax = plt.subplots(2,1,figsize=(5,8), sharex=True, constrained_layout=True)
ax[0].set_ylabel('mean melt rate [m/yr]', fontsize=14)
ax[1].set_ylabel('sensitivity [m/yr/K]' , fontsize=14)
ax[1].set_xlabel(r'ambient temperature [$^\circ$C]', fontsize=14)
Tas = np.arange(-2,2.5,.5)
testcase = 'test3'
# ref = ax[0].axvspan(-.15,.15, alpha=.1, color='purple', label='reference')
sm  = ax[0].scatter(np.arange(-2,2.1,.5), sheet[1], c='k', label='Sheet')
ax[1].plot(Tas, np.gradient(sheet[1], .5), c='k')
lb, = ax[0].plot(Tas,  7*(Tas+2), c='lightgrey', label='7 m/yr/K')
ub, = ax[0].plot(Tas, 16*(Tas+2), c='darkgrey', label='16 m/yr/K')
ax[1].plot(Tas,  7*np.ones_like(Tas), c='lightgrey')
ax[1].plot(Tas, 16*np.ones_like(Tas), c='darkgrey' )
handles = []
for j, model in enumerate(models[:-1]):
    rates = np.zeros_like(Tas)
    for t, T in enumerate(Tas):
        if model=='Plume':
            c = 'C0'
            pdict={'Ta':T}
            ds = IdealGeometry(name=testcase, pdict=pdict).create()
            results = PlumeModel(ds).compute_plume()
            melt = results.m.mean().values
        elif model=='PICO':
            c = 'C1'
            _, results = PicoModel(name=testcase, Ta=T).compute_pico()
            melt = results.melt.mean().values
        elif model=='PICOP':
            c = 'C2'
            _, _, results = PicoPlumeModel(name=testcase, Ta=T, n=3).compute_picop()
            melt = results.m.mean().values
        rates[t] = melt
        s = ax[0].scatter(T, melt, c=c, label=model)
    handles.append(s)
    ax[1].plot(np.arange(-2,2.1,.5), np.gradient(rates, .5), c=c)
ax[0].legend(handles=handles[:3]+[sm,lb,ub], fontsize=12)
ax[0].text(2,0, 'constant in-situ temp. & salinity', ha='right')
ax[0].text(2,5, 'geometry = `test3`', ha='right')
f.align_ylabels()

In [ ]:
rates

In [ ]:
plt.plot(np.arange(-2,2.1,.5), np.gradient(sheet[1], .5))


## tanh Ta/Sa profiles

In [ ]:
from ideal_geometry import Forcing

In [ ]:
zts = np.arange(-700,-200,50)
Tds = np.arange(-2,2.5,.5)
TT, ZZ = np.meshgrid(Tds, zts)
Mavg = np.empty((len(Tds),len(zts)))
Mmax = np.empty((len(Tds),len(zts)))

In [ ]:
plt.imshow(ZZ)
plt.colorbar()

In [ ]:
ds = xr.Dataset(coords={'z':np.arange(-5000,0,1)})

In [ ]:
ds

In [ ]:
Forcing().create()

In [ ]:
models

In [ ]:
%%time
for j, model in enumerate(models[:2]):
    print(j)
    f, ax = plt.subplots(2,2,figsize=(8,8), sharey=True, constrained_layout=True)
    f.suptitle(model)
    ax[0,0].set_ylabel('mean melt rate [m/yr]\nthermocline depth [m]')
    ax[1,0].set_ylabel('max. melt rate [m/yr]\nthermocline depth [m]')
    for i, testcase in enumerate(['test1', 'test3']):
        ax[1,i].set_xlabel(r'ambient temperature [$^\circ$C]')
        ax[0,i].set_title(testcase)
        
        for t, T in enumerate(Tds):
            for z, Z in enumerate(zts):
                ds = IdealGeometry(name=testcase, pdict=dict(Tdeep=T, ztcl=Z, name=testcase)).create()
                if model=='Plume':
                    results = PlumeModel(ds).compute_plume()
                    m = 'm'
                elif model=='PICO':
                    _, results = PicoModel(ds=ds).compute_pico()
                    m = 'melt'
                elif model=='PICOP':
                    _, _, results = PicoPlumeModel(name=testcase, Ta=T, n=3).compute_picop()
                    m = 'm'
                melt = results[m].mean().values
                mmax = results[m].max().values
                
            Mavg[t,z] = melt
            Mmax[t,z] = mmax
        ax[0,i].pcolormesh(TT, ZZ, Mavg.T)
        ax[1,i].pcolormesh(TT, ZZ, Mmax.T)

In [ ]:
plt.pcolormesh(TT, ZZ, Mavg.T, vmin=0, vmax=100)


In [ ]:
plt.pcolormesh(TT, ZZ, Mmax.T, vmin=0, vmax=100)

In [ ]:
plt.imshow(Mavg, vmin=0, vmax=100)
plt.colorbar()

In [ ]:
f, ax = plt.subplots(1,2,figsize=(10,4), sharey=True, constrained_layout=True)


In [ ]:
# pdict = dict(Ta=0, Sa=34)
pdict = dict(ztcl=-700, Tdeep=0)

ds = IdealGeometry(name=testcase, pdict=pdict).create()
ds

In [ ]:
ds.Tz.plot()

In [ ]:
ds.Ta.plot()

In [ ]:
ds.Sa.plot()

In [ ]:
d = xr.Dataset(coords={'z':np.arange(-5000,0,1)})

In [ ]:
d.assign_coords({'x':np.arange(4)})